<h1 align=center><font size = 5>Scraping Wikipedia Data</font></h1>

### Install Beautiful Soup package

In [1]:
!pip install bs4

### Import libraries

In [2]:
from bs4 import BeautifulSoup
import pandas as pd

### Download the wikipedia url

In [3]:
!wget https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M -O toronto_postalcodes.html

--2021-06-25 04:19:55--  https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
Resolving en.wikipedia.org (en.wikipedia.org)... 198.35.26.96, 2620:0:863:ed1a::1
Connecting to en.wikipedia.org (en.wikipedia.org)|198.35.26.96|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 83382 (81K) [text/html]
Saving to: ‘toronto_postalcodes.html’

toronto_postalcodes 100%[===================>]  81.43K   457KB/s    in 0.2s    

2021-06-25 04:19:55 (457 KB/s) - ‘toronto_postalcodes.html’ saved [83382/83382]



### Read the wikipedia html data

In [4]:
html = ''.join(open("toronto_postalcodes.html").readlines())

### Process wikipedea data into dataframe

In [5]:
soup = BeautifulSoup(html, 'html.parser')
table_contents=[]
table = soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = row.span.text.split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

In [42]:
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [43]:
df.shape

(103, 3)

### Read geospatial coordinates data

In [44]:
coords = pd.read_csv('./Geospatial_Coordinates.csv')

In [50]:
coords.columns = ['PostalCode', 'Latitude', 'Longitude']
coords = coords.merge(df)

In [51]:
coords.head()

,PostalCode,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae
